## Yield calculator.

This notebook contains the code to calculate yields from e.g. Battino et a. (2019, 2021) or Ritter et al. (2018) MESA models + stellar postprocessing.

The yields we use here are the integrated total ejected mass yield, i.e.

$$
EM = \int X_{i, \rm surf}(t)\,\dot{M}(t)\ dt
$$

To calculate this integral, we simply sum over the surface abundance (times the change in mass) for each timestep of the star. We additionally add the mass contained outside the final reminant at the final snapshot time as this should be included as well.

In [ ]:
from nugridpy import mesa as ms
from nugridpy import nugridse as mp

# default data location (try mirror if default
# is not available)
import matplotlib.pyplot as plt

In [ ]:
import sys, contextlib, io

In [ ]:
import numpy as np

### The MESA stellar evolution model

By default MESA is putting out two types of data. History data provides the time evolution of scalar quantities, one per time step. This data can be accessed with the `mesa.star_log` (or `mesa.history_data` which is the same) class.

MESA also outputs profile data at select time steps. Profiles are available via the `mesa_profile` class.

#### History data
Initialise the 2 solar-mass Z=0.02 MESA stellar evolution model from set1.2 using the seeker method:

In [ ]:
# is not available)
data_dir="/data/ASDR/NuGrid"  

# mirror NuGrid data location - uncomment both of the following lines
# ![ ! -h /user/data ] && ln -s /data/nugrid_data /user/data
# data_dir = '/user'

ms.set_nugrid_path(data_dir)
mp.set_nugrid_path(data_dir)

In [ ]:
setname = "set1.2"
modelname = "M2.00Z2.0e-02"

In [ ]:
pt = mp.se(sedir=f"/data/nugrid_data/set1ext/{setname}/ppd_wind/{modelname}/H5_out")



In [ ]:
pt_surf = mp.se(sedir=f"/data/nugrid_data/set1ext/{setname}/ppd_wind/{modelname}/H5_surf/")

#### What quantities are available in this data type, and what are the units?
Each of the _se_ file sets (in fact each of the dozens of hdf5 files that make up the data set for one mass/metallicty combination, or stellar evolution track) has three types of data contained in them:

data type access | content 
----------------|---------
 `pt.se.hattrs` |  a header section that holds the _header attributes_, including units in the form of factors so that if applied with the quantities the result is in cgs units 
`pt.se.cattrs` | for each cycle (or time step) the _cycle attributes_ are a number of scalar global quantities, such as total mass or star age
`pt.se.dcols` | again, for each time step these are the vector quantities available, i.e. the data table columns; one of the data columns, _iso_massf_ is in fact a matrix where the matrix columns are different species, i.e. a radial vector of species vectors

In [ ]:
pt.se.hattrs

In [ ]:
pt.se.cattrs

In [ ]:
pt.se.dcols

In [ ]:
def calc_wind_yields(mass, iso_massf):
    EM = np.zeros(len(iso_massf[0]))
    dm = -np.diff(mass)

    
    for i in range(len(mass) - 1):
        print(f"{i} / {len(mass)}", end="\r")
        X = iso_massf[i + 1]
        EM += dm[i] * X
        
    return EM
    

In [ ]:
def get_setname(modelname):
    Z = modelname.split("Z")[1]
    Z = float(Z)
    if Z == 0.02:
        setname = "set1.2"
    elif Z == 0.01:
        setname = "set1.1"
    elif Z == 6.0e-03:
        setname = "set1.3a"
    elif Z == 1.0e-03:
        setname = "set1.4a"
    elif Z == 1.0e-04:
        setname = "set1.5a"
    else:
        raise Exception("set not found")
    return setname

In [ ]:
def calc_rem_yield(pt, m_rem):
    cyclemax = np.int64(pt.se.cycles[-1])
    mass = pt.get(cyclemax, "mass")
    filt = mass > m_rem
    
    f = io.StringIO()
    Xs = pt.get(cyclemax, "iso_massf")[filt, :]
    
    dm = np.gradient(mass)[filt]
    
    ele_mass = Xs * np.reshape(dm, (-1, 1))

    
    return np.sum(ele_mass, axis=0)

In [ ]:
def calc_all(modelname, *, H_min=0.01, tol=1e-3):
    setname = get_setname(modelname)
    print("reading ", setname)
    pt = mp.se(sedir=f"/data/nugrid_data/set1ext/{setname}/ppd_wind/{modelname}/H5_out/")
    pt_surf = mp.se(sedir=f"/data/nugrid_data/set1ext/{setname}/ppd_wind/{modelname}/H5_surf/")
    print("read all files")
    
    isotopes = pt.se.isotopes
    print(isotopes[0:3])
    idx_h = np.where(np.array(pt.se.isotopes) == "H-1")[0][0]

    assert isotopes[idx_h] == "H-1" # need for later...

    print("reading arrays")
    Xs_surf = pt_surf.get("iso_massf")
    mass = pt_surf.get("mass")
    
    print("calculating wind yields")
    EM_wind = calc_wind_yields(mass, Xs_surf)
    m_ej = mass[0] - mass[-1]
    
    if abs(1 - m_ej / np.sum(EM_wind)) > tol:
        print("mass mismatch")
        print("yield sum: ", EM_wind)
        print("expected", m_ej)
        
    
    print("calculating reminant yields")
    cyclemax = np.int64(pt.se.cycles[-1])
    m_end = pt.get(cyclemax, "mass")
    Xs_end = pt.get(cyclemax, "iso_massf")
    
    m_rem = m_end[Xs_end[:, 0] > H_min][0]
    
    rem_yield = calc_rem_yield(pt, m_rem)
    
    tot_yield = rem_yield + EM_wind
    
    if abs(1 - (np.sum(tot_yield) + m_rem)/mass[0]) > tol:
        print("mass loss from yields ", np.sum(tot_yield))
        print("mini - mrem", mass[0] - m_rem)
    
    
    filename = f"yields_{modelname}.txt"
    Xi = Xs_surf[0]

    if np.abs(np.sum(Xi)-1) > tol:
        print("surface initial abundance sum:", np.sum(Xi))
    print("writing to file ", filename)

    write_header(filename, modelname, setname, mass, m_rem)
    write_yields(filename, isotopes, tot_yield, Xi)
            
    return tot_yield

In [ ]:
modelname

In [ ]:
from datetime import datetime

In [ ]:
def write_header(filename, modelname, setname, mass, m_rem):
    with open(filename, "w") as file:
        print("# Ritter 2018 yields", file=file)
        print(f"# Model: {modelname}", file=file)
        print(f"# Set: {setname}", file=file)
        print(f"# Mini: {mass[0]:0.2f}", file=file)
        print(f"# Mfinal: {m_rem}", file=file)
        print(f"# Created by Daniel Boyea on {datetime.today().strftime('%Y-%m-%d')}", file=file)
        print(f"{'isotope':10}    {'mass_yield':8}     {'X0':8}", file=file)

In [ ]:
def write_yields(filename, isotopes, yields, Xini):
    with open(filename, "a") as file:
        for i in range(len(tot_yield)):
            print(f"{isotopes[i]:10}     {yields[i]:8.4e}     {Xini[i]:8.4e}", file=file)

In [ ]:
calc_all("M2.00Z2.0e-02")

In [ ]:
calc_all("M7.000Z0.0010")

In [ ]:
calc_all("M5.00Z2.0e-02")

In [ ]:
Xs_surf = pt_surf.get("iso_massf")

In [ ]:
mass = pt_surf.get("mass")

In [ ]:
EM_wind = calc_wind_yields(mass, Xs_surf)

In [ ]:
mass[0] - mass[-1]

In [ ]:
np.sum(EM_wind)

In [ ]:
cyclemax = np.int64(pt.se.cycles[-1])

In [ ]:
m_end = pt.get(cyclemax, "mass")

In [ ]:
m_end[-1]

In [ ]:
Xs_end = pt.get(cyclemax, "iso_massf")

In [ ]:
m_rem = m_end[Xs_end[:, 0] > 0.01][0]

In [ ]:
m_rem

In [ ]:
rem_yield = calc_rem_yield(pt, m_rem)

In [ ]:
isotopes = (pt.se.isotopes)

In [ ]:
np.sum(rem_yield)

In [ ]:
tot_yield = rem_yield + EM_wind

In [ ]:
np.sum(tot_yield) + m_rem

In [ ]:
modelname

In [ ]:
print(f"{'Element':10}\t{'total':8}\t{'ejectedonly'}")
for i in range(len(tot_yield)):
    print(f"{isotopes[i]:10}\t{tot_yield[i]:8.4e}\t{EM_wind[i]:8.4e}")
    
    

## Surface abundance plots

In [ ]:
model_number = [int(c) for c in pt_surf.se.cycles]


In [ ]:
Xs_surf = np.array(pt_surf.get(model_number, "iso_massf"))

In [ ]:
m_star = pt_surf.get("mass")

In [ ]:
isotopes = np.array(pt_surf.se.isotopes)

In [ ]:
Xs.shape

In [ ]:
model_number = np.int64(model_number)

In [ ]:
model_number

In [ ]:
for iso in isotopes:
    print(iso)

In [ ]:
sys.path.append("../arya")
import arya

In [ ]:
#ifig=124;close(ifig);figure(ifig)



def plot_isos(isos, relative=True):
    for (i, ele) in enumerate(isos):
        ii = np.where(isotopes == ele)[0]
        if len(ii) == 1:
            i = ii[0]
            X = Xs[:, i]
            if relative:
                log_X0 = np.log10(X[0])
            else:
                log_X0=0
            
            y = np.log10(X) - log_X0
            plt.plot(model_number, y, label=ele,
                     ls  = ["-", "--", ":", "-."][i % 4]
                    )
        else:
            print("element not found", ele)

    plt.legend(bbox_to_anchor=(1, 1), loc="upper left")
    plt.xlabel("model number")
    plt.ylabel("log X / Xi")
    plt.show()

In [ ]:
isos = ["C-12", "C-13", "O-16", "Pb-206"]
plot_isos(isos)

In [ ]:
isos = ["Al-27", "P-31", "K-39"]
plot_isos(isos)

In [ ]:
isos = ["S-32", "Cl-35", "Ar-36", "Ca-40", "V-51", "Fe-56"]
plot_isos(isos)


In [ ]:
10**-0.003

In [ ]:
isos = ["Co-59", "Zn-64"]
plot_isos(isos)


In [ ]:
len(Xs[0])

In [ ]:
Xs[0] == Xs[300]